# BAC0

BAC0 is a script module based on BACpypes to automate task using Bacnet/IP protocole.

It's still at an early stage but will grow with time.

## How to use

First you must install the 2 modules
- git clone https://github.com/ChristianTremblay/bacpypes-python3.git
- git clone https://github.com/ChristianTremblay/bacnetScript.git

Both are installed using python setup.py install command in the shell
As you can see, it uses BACpypes-python3 which i a port from the official BACpypes Repo which is Python2 (http://bacpypes.sourceforge.net/)

Note : All tests have been done in Windows 7 using Anaconda3 (http://continuum.io/downloads)

## Import module

In [1]:
import BAC0

# You will need to define your local IP address (which is in the same subnet than the devices you're communicating with.
myIPAddr = '192.168.210.63'

bacnet = BAC0.ReadWriteScript(localIPAddr = '192.168.210.63')

# Wait for the app to start


DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:__init__ (<bacpypes.app.LocalDeviceObject object at 0x0000000005060FD0>, '192.168.210.63')
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:__init__ (<bacpypes.app.LocalDeviceObject object at 0x0000000005060FD0>, '192.168.210.63')
DEBUG:BAC0.scripts.BasicScript:finally
DEBUG:BAC0.core.functions.WhoisIAm.WhoisIAm:do_whois 'any'
DEBUG:BAC0.core.functions.WhoisIAm.WhoisIAm:    - request: <bacpypes.apdu.WhoIsRequest(8) instance at 0x779ef28>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:request <bacpypes.apdu.WhoIsRequest(8) instance at 0x779ef28>


Starting app...
App started


DEBUG:BAC0.scripts.BasicScript:finally
DEBUG:BAC0.core.functions.WhoisIAm.WhoisIAm:do_whois 'any'
DEBUG:BAC0.core.functions.WhoisIAm.WhoisIAm:    - request: <bacpypes.apdu.WhoIsRequest(8) instance at 0x779ef28>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:request <bacpypes.apdu.WhoIsRequest(8) instance at 0x779ef28>


Starting app...
App started


DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:do_IAmRequest <bacpypes.apdu.IAmRequest(0) instance at 0x77eec50>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:received broadcast from 192.168.210.253 (local:('192.168.210.63', 47808) | source:192.168.210.253)

DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:do_IAmRequest <bacpypes.apdu.IAmRequest(0) instance at 0x77eec50>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:received broadcast from 192.168.210.253 (local:('192.168.210.63', 47808) | source:192.168.210.253)

DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:do_IAmRequest <bacpypes.apdu.IAmRequest(0) instance at 0x77eee48>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:received broadcast from 2:5 (local:('192.168.210.63', 47808) | source:2:5)

DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:do_IAmRequest <bacpypes.apdu.IAmRequest(0) instance at 0x77eee48>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:received bro

## Using the application

The application is to be used to test bacnet devices. As it uses BACpypes, there's a lot of different functions that are available (whois, Iam, read, readMultiple, write, etc.) Actually, the basic scripts have been defined to be able to find devices, find points, read to points and write to points.

If you ever want to stop the app and free the socket, you must use :

**bacnet.stopApp()**

If not, you will need to restart the kernel...

### Finding devices on a network

In [2]:
bacnet.whois() # Note that this function is called automatically in ReadWriteScript

# The result takes a few milliseconds to be processed... it can be used in the same cell.

DEBUG:BAC0.core.functions.WhoisIAm.WhoisIAm:do_whois 'any'
DEBUG:BAC0.core.functions.WhoisIAm.WhoisIAm:    - request: <bacpypes.apdu.WhoIsRequest(8) instance at 0x5060780>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:request <bacpypes.apdu.WhoIsRequest(8) instance at 0x5060780>


defaultdict(<class 'int'>, {})

DEBUG:BAC0.core.functions.WhoisIAm.WhoisIAm:do_whois 'any'
DEBUG:BAC0.core.functions.WhoisIAm.WhoisIAm:    - request: <bacpypes.apdu.WhoIsRequest(8) instance at 0x5060780>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:request <bacpypes.apdu.WhoIsRequest(8) instance at 0x5060780>


defaultdict(<class 'int'>, {})

DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:do_IAmRequest <bacpypes.apdu.IAmRequest(0) instance at 0x77c3128>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:received broadcast from 192.168.210.253 (local:('192.168.210.63', 47808) | source:192.168.210.253)

DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:do_IAmRequest <bacpypes.apdu.IAmRequest(0) instance at 0x77c3128>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:received broadcast from 192.168.210.253 (local:('192.168.210.63', 47808) | source:192.168.210.253)

DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:do_IAmRequest <bacpypes.apdu.IAmRequest(0) instance at 0x77c3240>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:received broadcast from 2:5 (local:('192.168.210.63', 47808) | source:2:5)

DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:do_IAmRequest <bacpypes.apdu.IAmRequest(0) instance at 0x77c3240>
DEBUG:BAC0.core.app.ScriptApplication.ScriptApplication:received bro

In [3]:
# Result can be accessed to print with : 
for each in bacnet.discoveredDevices:
    print(each)


('192.168.210.253', 210253)
('2:5', 5)


This may be not clear enough... what if we would want to know the name of the devices...

In [4]:
# Result can be accessed to print with : 
print('==========================================')
print('Result of the Whois request')
print('==========================================')
for each in bacnet.discoveredDevices:
    deviceName = (bacnet.read('%s device %s objectName' % (each[0], each[1])))
    deviceAddr = each[0]
    print('Found device : %s at address %s' % (deviceName, deviceAddr))

Result of the Whois request
Found device : BASRT-B009157 at address 192.168.210.253
Found device : FX14 0005 at address 2:5


### Read objects from devices

Accessing the list of object of a device can be done using the objectList property


In [5]:
# Let's print the first 10 objects...
bacnet.read('2:5 device 5 objectList')[:10]

[('file', 1),
 ('analogInput', 2),
 ('analogInput', 3),
 ('analogInput', 5),
 ('analogInput', 4),
 ('analogInput', 0),
 ('analogInput', 1),
 ('analogOutput', 0),
 ('analogValue', 28),
 ('analogValue', 29)]

In [6]:
bacnet.readMultiple('2:5 analogInput 1 objectName description presentValue units')

['nvoAI2', 'nvoAI2', 22.760000228881836, 'degreesCelsius']

Doesn't tell you a lot of details... we will have to read values to get them. To accelerate the process, we'll use the special function "discoverPoints" which will return a tuple with a lot of details... and a pandas DataFrame, a good way to show table values.

In [7]:
from BAC0.core.functions.discoverPoints import discoverPoints
r = discoverPoints(bacnet,('2:5'),5)

Found FX14 0005


In [8]:
# r is now a tuple with device name, protocole services supported array, objectList and a DataFrame
# Let's print the head of the DataFrame, you can see that variables have been read

pointsTable = r[3]

pointsTable.head()

,pointType,pointAddress,description,presentValue,units
pointName,,,,,
nvoAI3,analogInput,2,nvoAI3,13.68,degreesCelsius
nvoAI4,analogInput,3,nvoAI4,48.465,percent
nvoAI6,analogInput,5,nvoAI6,393,kilopascals
nvoAI5,analogInput,4,nvoAI5,1.85,percent
nvoAI1,analogInput,0,nvoAI1,23.36,degreesCelsius


In [9]:
print(pointsTable.ix['nvoAI3'].pointType)

analogInput


In [10]:
def readPointByName(name):
    device = '2:5'
    val = bacnet.read('%s %s %s presentValue' % (device,pointsTable.ix[name].pointType,str(pointsTable.ix[name].pointAddress)))
    return val

readPointByName('nvoAI4')

48.4900016784668

## Write to points

In [2]:
import time
obj = '2:5 analogValue 41 presentValue'
old_value = bacnet.read(obj)
print('Old Value is : %s' % old_value)
new_value = 120

bacnet.write('%s %s' % (obj,new_value))
print('New value is %s' % bacnet.read(obj))

Old Value is : 120.0
New value is 120.0


## Stopping the app

In [11]:
bacnet.stopApp()

DEBUG:bacpypes.core:stop


Stopping app
App stopped


In [2]:
task = BAC0.RecurringRead(bacnet,1,('2:5 analogInput 2 presentValue'),5)

In [1]:
point_list = [
    ('2:5', 'analogInput', 1, 'presentValue'),
    ('2:5', 'analogInput', 2, 'presentValue'),
    ]


In [25]:
from threading import Thread, Lock
import time

class repeat(Thread):
    def __init__(self,threadID,name, req1, req2, lock):
        Thread.__init__(self)
        self.threadID = threadID
        self.name = name
        self.request1 = req1
        self.request2 = req2
        self.exitFlag = False
        self.lock = lock
    
    def run(self):
        # Get lock to synchronize threads
        print("Starting " + self.name)
        #print_time(self.name)
        self.do_it()
        try:
            self.lock.release()
        except Exception:
            pass
        print("Exiting " + self.name)
    
    def do_it(self):
        #if self.started = True
        while not self.exitFlag:
            self.lock.acquire()
            res1 = bacnet.read(self.request1)
            print('Result1 of %s : %s' % (self.name,res1))
            res2 = bacnet.read(self.request2)
            print('Result2 of %s : %s' % (self.name,res2))
            # Free lock to release next thread
            self.lock.release()
            time.sleep(5)

threadLock = Lock()
thread1 = repeat(1, "Thread-1",('2:5 analogInput 1 presentValue'),('2:5 analogInput 2 presentValue'),threadLock)
thread2 = repeat(2, "Thread-2",('2:5 analogInput 3 presentValue'),('2:5 analogInput 4 presentValue'),threadLock)

# Start new Threads
thread1.start()
thread2.start()
    

Exiting Thread-2
Starting Thread-1
Starting Thread-2
Result1 of Thread-1 : 23.020000457763672
Result2 of Thread-1 : 12.739999771118164
Result1 of Thread-2 : 48.63999938964844
Result2 of Thread-2 : 1.850000023841858
Result1 of Thread-1 : 23.059999465942383
Result2 of Thread-1 : 12.729999542236328


In [26]:
thread1.exitFlag = True

Result1 of Thread-2 : 48.63999938964844
Result2 of Thread-2 : 1.850000023841858
Exiting Thread-1
Result1 of Thread-2 : 48.650001525878906


In [27]:
thread2.exitFlag = True

Result2 of Thread-2 : 1.850000023841858


In [28]:
thread3 = repeat(3, "Thread-3",('2:5 analogInput 5 presentValue'),('2:5 analogInput 1 presentValue'),threadLock)
thread3.start()

Exiting Thread-2
Starting Thread-3
Result1 of Thread-3 : 357.8999938964844
Result2 of Thread-3 : 23.020000457763672
Result1 of Thread-3 : 357.8999938964844
Result2 of Thread-3 : 23.020000457763672


In [29]:
thread3.exitFlag = True